In [195]:
import pandas as pd
from collections import Counter

In [196]:
sda1 = pd.read_excel(r'D:\coding\coding semester 1\coding semester 2\UAS\SDA 1.xlsx')

In [197]:
#CLASS MEMBANGUN TREENODE
class TreeNode:
    def __init__(self,nama = '', kondisi = None, jurusan = None):
        self.nama = nama
        self.kondisi = kondisi
        self.jurusan = jurusan
        self.anak = []
        
    def tambah_anak(self,Node):
        self.anak.append(Node)
        
    def cek_jurusan(self, skor_LP, skor_LB, skor_PU):
        hasil_jurusan = []
        if self.jurusan :
            return [self.jurusan]
        for anak in self.anak:
            if anak.kondisi is None or anak.kondisi(skor_LP, skor_LB, skor_PU):
                hasil_jurusan.extend(anak.cek_jurusan (skor_LP, skor_LB, skor_PU))
        return hasil_jurusan
            
            


In [198]:
# ALGORITMA SORTING UNTUK SLEKSI JURUSAN
def seleksi_jurusan_DT1(posisi_LP, posisi_LB,posisi_PU ,skor_LP, skor_LB ,skor_PU):
    if posisi_LP == 1 and skor_LP >= 650:
        return (posisi_LB == 2 and skor_LB >= 600 ) or (posisi_PU == 2 and skor_PU >= 600)
    elif posisi_LP == 2 and 500 <= skor_LP < 650:
        return (posisi_LB == 1 and skor_LB >= 600) or (posisi_PU == 1 and skor_PU < 600)
    elif posisi_LP == 3 and skor_LP < 500:
        return (posisi_LB == 1 and skor_LB >= 650 and posisi_PU == 2 and skor_PU >= 600) or (posisi_LB == 2 and skor_LB >= 600 and posisi_PU == 1 and skor_PU >650)
    return False

def seleksi_jurusan_DT2(posisi_LB, posisi_LP, posisi_PU, skor_LB, skor_LP, skor_PU):
    if posisi_LB == 1 and skor_LB >= 650:
        return (posisi_LP == 2 and skor_LP >= 600) or (posisi_PU == 2 and skor_PU >= 600)
    elif posisi_LB == 2 and 500 <= skor_LB < 650:
        return (posisi_LP == 1 and skor_LP >= 600) or (posisi_PU == 1 and skor_PU < 600)
    elif posisi_LB == 3 and skor_LB < 500:
        return (posisi_LP == 1 and skor_LP >= 650 and posisi_PU == 2 and skor_PU >= 600) or (posisi_LP == 2 and skor_LP >= 600 and posisi_PU == 1 and skor_PU > 650)
    return False

def seleksi_jurusan_DT3(posisi_PU, posisi_LB, posisi_LP, skor_PU, skor_LB, skor_LP):
    if posisi_PU == 1 and skor_PU >= 650:
        return (posisi_LB == 2 and skor_LB >= 600) or (posisi_LP == 2 and skor_LP >= 600)
    elif posisi_PU == 2 and 500 <= skor_PU < 650:
        return (posisi_LB == 1 and skor_LB >= 600) or (posisi_LP == 1 and skor_LP < 600)
    elif posisi_PU == 3 and skor_PU < 500:
        return (posisi_LB == 1 and skor_LB >= 650 and posisi_LP == 2 and skor_LP >= 600) or (posisi_LB == 2 and skor_LB >= 60 and posisi_LP == 1 and skor_LP >= 650)
    return False


In [199]:
#PASANG ROOT DAN MEMBUAT TREE
def build_treenode_DT(sda1,prio='LP'):
    root = TreeNode('root')
    for _, row in sda1.iterrows():
        prioritas = (row['prioritas 1'], row['prioritas 2'], row['prioritas 3'])
        if set(prioritas) != {'LP', 'LB', 'PU'}:
            continue
        
        posisi_LP = prioritas.index('LP') + 1
        posisi_LB = prioritas.index('LB') + 1
        posisi_PU = prioritas.index('PU') + 1
        
        def seleksi_jurusan(sp_LP, sp_LB, sp_PU, pLP=posisi_LP, pLB=posisi_LB, pPU=posisi_PU):
            if prio == 'LP':
                return seleksi_jurusan_DT1(pLP, pLB,pPU ,sp_LP, sp_LB ,sp_PU)
            elif prio == 'LB':
                return seleksi_jurusan_DT2(pLB, pLP, posisi_PU, sp_LB, sp_LP, sp_PU)
            elif prio == 'PU':
                return seleksi_jurusan_DT3(posisi_PU, pLB, pLP, sp_PU, sp_LB, sp_LP)
            return False
        jur_node = TreeNode(nama = row['Jurusan'],kondisi = seleksi_jurusan,jurusan = row['Jurusan'])
        root.tambah_anak(jur_node)
        
    return root
#GABUNG HASIL DAN SELEKSI MANA YANG DOBEL TERBANYAK    
def rekomendasi_jurusan(sda1, skor_LP, skor_LB, skor_PU):
    hasil_DT1 = build_treenode_DT(sda1,'LP'). cek_jurusan(skor_LP, skor_LB, skor_PU)
    hasil_DT2 = build_treenode_DT(sda1,'LB'). cek_jurusan(skor_LP, skor_LB, skor_PU)
    hasil_DT3 = build_treenode_DT(sda1,'PU'). cek_jurusan(skor_LP, skor_LB, skor_PU)
    
    gabungan = hasil_DT1 + hasil_DT2 + hasil_DT3
    counter = Counter(gabungan)   
    
    if counter :
        terbanyak = max(counter.values())
        rekomendasi_jurusan_final = [jurusan for jurusan,frekuensi in counter.items() if frekuensi == terbanyak]
        return rekomendasi_jurusan_final
    return []
       


In [200]:
#SELEKSI PTN BERDASARKAN NILAI RATA RATA  DAN PG
def seleksi_PTN(sda1, daftar_jurusan, nilai_total):
    hasil_ptn = {}
    for jurusan in daftar_jurusan:
        baris = sda1[sda1['Jurusan'] == jurusan]
        if not baris.empty:
            PTN = []
            for ptn in ['ITS', 'UNAIR', 'UB', 'UM', 'UNESA', 'UPNVJT']:
                pg = baris[ptn].values[0]
                if pd.notna(pg) and pg > 0 and nilai_total >= pg:
                    PTN.append(ptn)
            hasil_ptn[jurusan] = PTN
    return hasil_ptn

In [ ]:
# input nilai
nilai_PU = float(input("Masukkan nilai Pengetahuan Umum anda: "))
nilai_PPU = float(input("Masukkan nilai Pengetahuan Pemahaman Umum anda: ")) 
nilai_PBM = float(input("Masukkan nilai Pengetahuan Baca dan Menulis anda: ")) 
nilai_PK = float(input("Masukkan nilai Pengetahuan Kuatitatif anda: ")) 
nilai_LITBIN = float(input("Masukkan nilai Literasi dalam Bahasa Indonesiaanda: ")) 
nilai_LITBING = float(input("Masukkan nilai Literasi Dalam Bahasa Inggris anda: ")) 
nilai_PM = float(input("Masukkan nilai Penalaran Matematika  anda: ")) 

skor_LB = (nilai_LITBIN + nilai_LITBING)/2
skor_LP = (nilai_PK + nilai_PM)/2
skor_PU = (nilai_PU + nilai_PPU + nilai_PBM)/3
    


In [203]:
# semua jurusan ditampilkan
nilai_total = (skor_LP + skor_LB + skor_PU) / 3
hasil_akhir = rekomendasi_jurusan(sda1, skor_LP, skor_LB, skor_PU)
hasil_ptn = seleksi_PTN(sda1, hasil_akhir, nilai_total)

print(f"NILAI RATA-RATA = {nilai_total:.2f}")
print("Rekomendasi Jurusan dan PTN:")
for i,jurusan in enumerate(hasil_ptn,start = 1):
    print(f"{i}.{jurusan}")
    if jurusan in hasil_ptn and hasil_ptn[jurusan]:
        for ptn in hasil_ptn[jurusan]:
            print(f" -{ptn}")
    else:
        print("   (Tidak ada universitas yang memenuhi passing grade)")


NILAI RATA-RATA = 495.61
Rekomendasi Jurusan dan PTN:


In [204]:
#jurusan tanpa univ disembunyikan 
nilai_total = (skor_LP + skor_LB + skor_PU) / 3
hasil_akhir = rekomendasi_jurusan(sda1, skor_LP, skor_LB, skor_PU)
hasil_ptn = seleksi_PTN(sda1, hasil_akhir, nilai_total)

# Filter jurusan yang punya minimal 1 PTN lolos
hasil_ptn_filtered = {j: ptns for j, ptns in hasil_ptn.items() if ptns}

print(f"NILAI RATA-RATA = {nilai_total:.2f}")
if not hasil_ptn_filtered:
    print ("maaf nilaimu tidak dapat memenuhi pasing grade")
    print("tetap semangat dan jangan putus asa ")
else:
    print("Rekomendasi Jurusan dan PTN:")
    for i, (jurusan, daftar_ptn) in enumerate(hasil_ptn_filtered.items(), start= 1):
        print(f"{i}. {jurusan}")
        for ptn in daftar_ptn:
            print(f" - {ptn}")
                

NILAI RATA-RATA = 495.61
maaf nilaimu tidak dapat memenuhi pasing grade
tetap semangat dan jangan putus asa 


ROOT
│
├── Cek Prioritas LP
│   ├── posisi_LP == 1 DAN skor_LP >= 650:
│   │   ├── Cek Prioritas LB
│   │   │   └── posisi_LB == 2 DAN skor_LB >= 600:
│   │   │       └───> Jurusan [A] direkomendasikan (True)
│   │   └── posisi_LP == 2 DAN skor_LP >= 600:
│   │       └───> Jurusan [B] direkomendasikan (True)
│   ├── posisi_LP == 2 DAN (500 <= skor_LP < 650):
│   │   ├── Cek Prioritas LB
│   │   │   └── posisi_LB == 1 DAN skor_LB >= 650:
│   │   │       └───> Jurusan [C] direkomendasikan (True)
│   │   └── posisi_LP == 1 DAN skor_LP >= 650:
│   │       └───> Jurusan [D] direkomendasikan (True)
│   └── posisi_LP == 1 DAN skor_LP < 500:
│       ├── Cek Prioritas LB
│           ├── (posisi_LB == 1 DAN skor_LB >= 650) dan (posisi_LP == 2 DAN skor_LP >= 600):
│           │   └───> Jurusan [E] direkomendasikan (True)
│           └── (posisi_LP == 1 DAN skor_LP >= 650) dan (posisi_LB == 2 DAN skor_LB >= 600):
│               └───> Jurusan [F] direkomendasikan (True)
│
├── Cek Prioritas LB
│   ├── posisi_LB == 1 DAN skor_LB >= 650:
│   │   ├── Cek Prioritas LP
│   │   │   └── posisi_LP == 2 DAN skor_LP >= 600:
│   │   │       └───> Jurusan [G] direkomendasikan (True)
│   │   └── posisi_PU == 2 DAN skor_PU >= 600:
│   │       └───> Jurusan [H] direkomendasikan (True)
│   ├── posisi_LB == 2 DAN (500 <= skor_LB < 650):
│   │   ├── Cek Prioritas LP
│   │   │   └── posisi_LP == 1 DAN skor_LP >= 650:
│   │   │       └───> Jurusan [I] direkomendasikan (True)
│   │   └── posisi_PU == 1 DAN skor_PU >= 650:
│   │       └───> Jurusan [J] direkomendasikan (True)
│   └── posisi_LB == 3 DAN skor_LB < 500:
│       ├── Cek Prioritas LB
│           ├── (posisi_LP == 1 DAN skor_LP >= 650) dan (posisi_PU == 2 DAN skor_PU >= 600):
│           │   └───> Jurusan [E] direkomendasikan (True)
│           └── (posisi_PU == 1 DAN skor_PU >= 650) dan (posisi_LP == 2 DAN skor_LP >= 600):
│               └───> Jurusan [F] direkomendasikan (True)
│
└── Cek Prioritas PU
    ├── posisi_PU == 1 DAN skor_PU >= 650:
    │   ├── Cek Prioritas LB
    │   │   └── posisi_LB == 2 DAN skor_LB >= 600:
    │   │       └───> Jurusan [M] direkomendasikan (True)
    │   └── posisi_LP == 2 DAN skor_LP >= 600:
    │       └───> Jurusan [N] direkomendasikan (True)
    ├── posisi_PU == 2 DAN (500 <= skor_PU < 650):
    │   ├── Cek Prioritas LB
    │   │   └── posisi_LB == 1 DAN skor_LB >= 650:
    │   │       └───> Jurusan [O] direkomendasikan (True)
    │   └── posisi_LP == 1 DAN skor_LP >= 650:
    │       └───> Jurusan [P] direkomendasikan (True)
    └── posisi_PU == 3 DAN skor_PU < 500:
        ├── Cek Prioritas LB
        ├── (posisi_LB == 1 DAN skor_LB >= 650) dan (posisi_LP == 2 DAN skor_LP >= 600):
        │   └───> Jurusan [E] direkomendasikan (True)
        └── (posisi_LP == 1 DAN skor_LP >= 650) dan (posisi_LB == 2 DAN skor_LB >= 600):
            └───> Jurusan [F] direkomendasikan (True)
